In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [3]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluate:float

In [4]:


def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']

    pmt = f'Write a detailed blog on the topic - "{title}" using the following outline:\n{outline}'
    content = model.invoke(pmt).content

    state['content'] = content
    
    return state

In [5]:
def create_outline(state: BlogState) -> BlogState:
    title = state['title']

    pmt = f'Generate an outline for a blog on the topic - {title}'

    outline = model.invoke(pmt).content
    state['outline'] = outline
    return state

In [6]:
def evaluate(state: BlogState) -> BlogState:
    content = state['content']
    pmt = f'Rate the following blog content out of 10 for quality and depth:\n\n{content}\n\nRespond with just a number (e.g., 8.5)'
    score_text = model.invoke(pmt).content.strip()

    try:
        score = float(score_text)
    except ValueError:
        score = 0.0

    state['evaluate'] = score
    return state

In [7]:
graph = StateGraph(BlogState)

graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)
graph.add_node("evaluate", evaluate)

graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", "evaluate")
graph.add_edge("evaluate", END)

workflow = graph.compile()

In [8]:
# execution 

initial_state = {
    "title": "Rise of AI in India",
    "outline": "",
    "content": "",
    "evaluate": 0.0
}

ans = workflow.invoke(initial_state)

print("Outline:\n", ans['outline'])
print("\nBlog Content:\n", ans['content'])
print("\nScore (out of 10):", ans['evaluate'])

Outline:
 ## Blog Outline: The Rise of AI in India

**I. Introduction**
    * Hook: Start with a compelling statistic or anecdote showcasing AI's growing presence in India.  (e.g.,  investment figures, a specific AI application's success story)
    * Briefly define AI and its relevance in the modern world.
    * Thesis statement:  Highlight the rapid growth of AI in India, its drivers, challenges, and future potential.

**II. Drivers of AI Growth in India**
    * **Government Initiatives:** Discuss policies like the National AI Strategy, Digital India, and their impact on fostering AI development.
    * **Technological Talent Pool:**  Mention the large pool of skilled engineers and data scientists, universities producing AI graduates, and the growing startup ecosystem.
    * **Data Abundance:**  Highlight the vast amount of data generated in India (social media, e-commerce, etc.) fueling AI development and training.
    * **Investment and Funding:** Discuss the increasing investments f